# Layer 3: LLM Deep Analysis

### What We're Building:
- Use Claude for nuanced analysis
- Get reasoning, not just labels
- Identify violation TYPE and SEVERITY

In [ ]:
from snowflake.snowpark import Session

session = Session.builder.getOrCreate()
session.use_warehouse('COMPLIANCE_DEMO_WH')
session.use_database('COMPLIANCE_DEMO')
session.use_schema('ML')

print("Layer 3: Adding LLM intelligence...")

## Step 1: Select Emails for LLM Analysis

In [ ]:
flagged_count = session.sql("""
    SELECT COUNT(*) as cnt 
    FROM MODEL_PREDICTIONS_V1 
    WHERE PREDICTED_VIOLATION = 1
""").collect()[0]['CNT']

print(f"ML flagged {flagged_count:,} emails as suspicious")
print(f"\nRunning Claude on a sample of 5 for this demo...")

## Step 2: Run Claude Analysis

In [ ]:
session.sql("""
CREATE OR REPLACE TABLE COMPLIANCE_DEMO.ML.LLM_ANALYSIS AS
WITH flagged_emails AS (
    SELECT 
        p.EMAIL_ID,
        p.COMPLIANCE_LABEL as ACTUAL_LABEL,
        p.PREDICTED_VIOLATION as ML_FLAG,
        e.SUBJECT,
        e.BODY,
        e.SENDER_DEPT,
        e.RECIPIENT_DEPT
    FROM MODEL_PREDICTIONS_V1 p
    JOIN COMPLIANCE_DEMO.EMAIL_SURVEILLANCE.EMAILS e ON p.EMAIL_ID = e.EMAIL_ID
    WHERE p.PREDICTED_VIOLATION = 1
    LIMIT 5
)
SELECT 
    EMAIL_ID,
    ACTUAL_LABEL,
    ML_FLAG,
    SENDER_DEPT,
    RECIPIENT_DEPT,
    SUBJECT,
    BODY,
    
    SNOWFLAKE.CORTEX.COMPLETE(
        'claude-3-5-sonnet',
        CONCAT(
            $$You are a hedge fund compliance expert. Analyze this email for violations.

VIOLATION TYPES:
- INSIDER_TRADING: Sharing MNPI, trading tips before announcements
- CONFIDENTIALITY_BREACH: Leaking client data, proprietary info
- PERSONAL_TRADING: Undisclosed personal trades, front-running
- INFO_BARRIER_VIOLATION: Research/Trading wall breaches
- CLEAN: Normal business communication

Email from: $$, SENDER_DEPT, $$ to: $$, RECIPIENT_DEPT, $$
Subject: $$, SUBJECT, $$
Body: $$, LEFT(BODY, 1500), $$

Respond with JSON: {"is_violation": true/false, "confidence": 0.0-1.0, "category": "...", "reasoning": "one sentence explanation"}$$
        )
    ) AS CLAUDE_ANALYSIS
FROM flagged_emails
""").collect()

print("Claude analysis complete!")

## Step 3: View Claude's Reasoning

In [ ]:
results = session.sql("""
    SELECT 
        EMAIL_ID,
        ACTUAL_LABEL,
        SENDER_DEPT || ' -> ' || RECIPIENT_DEPT as COMMUNICATION,
        SUBJECT,
        BODY,
        CLAUDE_ANALYSIS
    FROM LLM_ANALYSIS
""").to_pandas()

print("\n" + "="*80)
print("CLAUDE'S COMPLIANCE ANALYSIS")
print("="*80)

for _, row in results.iterrows():
    print(f"\n--- Email: {row['EMAIL_ID'][:8]}... ---")
    print(f"Route: {row['COMMUNICATION']}")
    print(f"Subject: {row['SUBJECT']}")
    print(f"Body: {row['BODY'][:300]}...")
    print(f"Actual Label: {row['ACTUAL_LABEL']}")
    print(f"\nClaude says: {row['CLAUDE_ANALYSIS'][:400]}...")

## The Power: Reasoning + Categories

Claude provides:
1. **Classification** - Is it a violation?
2. **Confidence** - How sure is it?
3. **Category** - WHAT TYPE of violation?
4. **Reasoning** - WHY is it a violation?

Keywords can't do this. ML can't do this. Only LLMs can explain their reasoning.

## Layer 3 Complete

**What we built:**
- LLM analysis with Claude
- Structured JSON output
- Human-readable reasoning

**Next:** Combine ML + LLM into a tiered architecture →